## 스타벅스 매장명, 주소, 구 크롤링

In [36]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import Chrome
from bs4 import BeautifulSoup
driver = webdriver.Chrome(executable_path='../driver/chromedriver.exe')
driver.get("https://www.starbucks.co.kr/")

In [37]:
import warnings
warnings.simplefilter(action='ignore')

In [38]:
# 스토어 탭 클릭
store = driver.find_element(By.CSS_SELECTOR, "#gnb > div > nav > div > ul > li.gnb_nav03 > h2 > a")
store.click()

In [39]:
# 매장 찾기
search_store = driver.find_element(By.CSS_SELECTOR, "#container > section.store_bn1_section > article > div.store_bn1_btn > a")
search_store.click()

In [40]:
# 지역 검색
search_local = driver.find_element(By.CSS_SELECTOR, "#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a")
search_local.click()

In [41]:
# 도시 선택
select_city = driver.find_element(By.CSS_SELECTOR, "#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a")
select_city.click()

In [42]:
# 전체 탭 클릭
driver.find_element(By.CSS_SELECTOR, "#mCSB_2_container > ul > li:nth-child(1) > a").click()

In [43]:
# 매장 리스트 가져온 것 확인.
seoul_list_selenium = driver.find_elements(By.CSS_SELECTOR, '#mCSB_3_container ul li')
len(seoul_list_selenium), seoul_list_selenium

(603,
 [<selenium.webdriver.remote.webelement.WebElement (session="c4790574d0bd0d0f62a8805c21a38826", element="0873BD703BC6F9A8D58D6D9F240DBF26_element_124")>,
  <selenium.webdriver.remote.webelement.WebElement (session="c4790574d0bd0d0f62a8805c21a38826", element="0873BD703BC6F9A8D58D6D9F240DBF26_element_125")>,
  <selenium.webdriver.remote.webelement.WebElement (session="c4790574d0bd0d0f62a8805c21a38826", element="0873BD703BC6F9A8D58D6D9F240DBF26_element_126")>,
  <selenium.webdriver.remote.webelement.WebElement (session="c4790574d0bd0d0f62a8805c21a38826", element="0873BD703BC6F9A8D58D6D9F240DBF26_element_127")>,
  <selenium.webdriver.remote.webelement.WebElement (session="c4790574d0bd0d0f62a8805c21a38826", element="0873BD703BC6F9A8D58D6D9F240DBF26_element_128")>,
  <selenium.webdriver.remote.webelement.WebElement (session="c4790574d0bd0d0f62a8805c21a38826", element="0873BD703BC6F9A8D58D6D9F240DBF26_element_129")>,
  <selenium.webdriver.remote.webelement.WebElement (session="c4790574d

In [44]:
# BeatifulSoup 활용
from bs4 import BeautifulSoup

In [45]:
html = driver.page_source
dom = BeautifulSoup(html, 'html.parser')

In [46]:
seoul_list = dom.select('#mCSB_3_container ul li')
seoul_list[0]['data-name'], seoul_list[0]['data-lat'], seoul_list[0]['data-long'], seoul_list[0].select_one('p').text[:-9]

('역삼아레나빌딩', '37.501087', '127.043069', '서울특별시 강남구 언주로 425 (역삼동)')

In [47]:
import pandas as pd
from tqdm import tqdm_notebook

datas = []

for idx in tqdm_notebook(seoul_list):
    title = idx['data-name']
    address = idx.select_one('p').text[:-9]
    lat = idx['data-lat']
    lng = idx['data-long']
    datas.append({
        'title' : title,
        'address' : address,
        'lat' : lat,
        'lng' : lng 
    })
driver.close()
df = pd.DataFrame(datas)
df.tail()


  0%|          | 0/603 [00:00<?, ?it/s]

,title,address,lat,lng
598,사가정역,서울특별시 중랑구 면목로 310,37.579594,127.087966
599,상봉역,서울특별시 중랑구 망우로 307 (상봉동),37.59689,127.08647
600,묵동,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",37.615368,127.076633
601,양원역,서울특별시 중랑구 양원역로10길 3 (망우동),37.6066536267232,127.106359790053
602,중화역,서울특별시 중랑구 봉화산로 35,37.60170912407773,127.07841136432036


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 603 entries, 0 to 602
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    603 non-null    object
 1   address  603 non-null    object
 2   lat      603 non-null    object
 3   lng      603 non-null    object
dtypes: object(4)
memory usage: 19.0+ KB


In [52]:
# 구 컬럼 만들기

df['gu'] = df['address'][0].split(' ')[1]

for idx, rows in df.iterrows():
    rows['gu'] = df['address'][idx].split()[1]

In [56]:
df.head()

,title,address,lat,lng,gu
0,역삼아레나빌딩,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069,강남구
1,논현역사거리,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223,강남구
2,신사역성일빌딩,서울특별시 강남구 강남대로 584 (논현동),37.5139309,127.0206057,강남구
3,국기원사거리,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495,강남구
4,대치재경빌딩R,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583,강남구


In [57]:
df.to_csv('../data/starbucks.csv', encoding='utf-8')

In [59]:
starbucks_df = pd.read_csv('../data/starbucks.csv', index_col=0)
starbucks_df

,title,address,lat,lng,gu
0,역삼아레나빌딩,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069,강남구
1,논현역사거리,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223,강남구
2,신사역성일빌딩,서울특별시 강남구 강남대로 584 (논현동),37.513931,127.020606,강남구
3,국기원사거리,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495,강남구
4,대치재경빌딩R,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583,강남구
...,...,...,...,...,...
598,사가정역,서울특별시 중랑구 면목로 310,37.579594,127.087966,중랑구
599,상봉역,서울특별시 중랑구 망우로 307 (상봉동),37.596890,127.086470,중랑구
600,묵동,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",37.615368,127.076633,중랑구
601,양원역,서울특별시 중랑구 양원역로10길 3 (망우동),37.606654,127.106360,중랑구
